In [5]:
#Task 4 (a)

In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Sets
P = ["A", "B", "C", "D"]
W = ["W1", "W2", "W3"]
D = ["DC1", "DC2", "DC3", "DC4", "DC5", "DC6"]

supply = {"A":70, "B":90, "C":80, "D":60}
demand = {"DC1":50, "DC2":40, "DC3":60, "DC4":70, "DC5":50, "DC6":30}

cPW = {
    ("A","W1"):4, ("A","W2"):3, ("A","W3"):5,
    ("B","W1"):6, ("B","W2"):2, ("B","W3"):4,
    ("C","W1"):5, ("C","W2"):4, ("C","W3"):3,
    ("D","W1"):7, ("D","W2"):5, ("D","W3"):6
}

cWD = {
    ("W1","DC1"):3, ("W1","DC2"):2, ("W1","DC3"):5, ("W1","DC4"):4, ("W1","DC5"):6, ("W1","DC6"):7,
    ("W2","DC1"):1, ("W2","DC2"):4, ("W2","DC3"):2, ("W2","DC4"):3, ("W2","DC5"):5, ("W2","DC6"):6,
    ("W3","DC1"):5, ("W3","DC2"):3, ("W3","DC3"):4, ("W3","DC4"):2, ("W3","DC5"):3, ("W3","DC6"):4
}

xPW = {(p,w): LpVariable(f"xPW_{p}_{w}", lowBound=0) for p in P for w in W}
xWD = {(w,k): LpVariable(f"xWD_{w}_{k}", lowBound=0) for w in W for k in D}

model = LpProblem("SupplyChainOptimization", LpMinimize)

model += lpSum(cPW[(p,w)]*xPW[(p,w)] for p in P for w in W) + lpSum(cWD[(w,k)]*xWD[(w,k)] for w in W for k in D)


for p in P:
    model += lpSum(xPW[(p,w)] for w in W) <= supply[p], f"Supply_{p}"

for k in D:
    model += lpSum(xWD[(w,k)] for w in W) == demand[k], f"Demand_{k}"

for w in W:
    model += lpSum(xPW[(p,w)] for p in P) == lpSum(xWD[(w,k)] for k in D), f"WarehouseBalance_{w}"

model.solve()

print("Status:", LpStatus[model.status])
print("Total Cost:", value(model.objective))

print("\nOptimal Shipments from Production to Warehouses:")
for p in P:
    for w in W:
        val = xPW[(p,w)].varValue
        if val > 0:
            print(f"{p} -> {w}: {val}")

print("\nOptimal Shipments from Warehouses to Distribution Centres:")
for w in W:
    for k in D:
        val = xWD[(w,k)].varValue
        if val > 0:
            print(f"{w} -> {k}: {val}")


Status: Optimal
Total Cost: 1700.0

Optimal Shipments from Production to Warehouses:
A -> W1: 40.0
A -> W2: 30.0
B -> W2: 90.0
C -> W3: 80.0
D -> W2: 60.0

Optimal Shipments from Warehouses to Distribution Centres:
W1 -> DC2: 40.0
W2 -> DC1: 50.0
W2 -> DC3: 60.0
W2 -> DC4: 70.0
W3 -> DC5: 50.0
W3 -> DC6: 30.0


In [3]:
#Task 4 (b)

In [7]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Sets
P = ["A", "B", "C", "D"]
W = ["W1", "W2", "W3"]
D = ["DC1", "DC2", "DC3", "DC4", "DC5", "DC6"]

# Supply and Demand
supply = {"A":70, "B":90, "C":80, "D":60}
demand = {"DC1":50, "DC2":40, "DC3":60, "DC4":70, "DC5":50, "DC6":30}

# Costs from Production to Warehouse (unchanged)
cPW = {
    ("A","W1"):4, ("A","W2"):3, ("A","W3"):5,
    ("B","W1"):6, ("B","W2"):2, ("B","W3"):4,
    ("C","W1"):5, ("C","W2"):4, ("C","W3"):3,
    ("D","W1"):7, ("D","W2"):5, ("D","W3"):6
}


cWD_original = {
    ("W1","DC1"):3, ("W1","DC2"):2, ("W1","DC3"):5, ("W1","DC4"):4, ("W1","DC5"):6, ("W1","DC6"):7,
    ("W2","DC1"):1, ("W2","DC2"):4, ("W2","DC3"):2, ("W2","DC4"):3, ("W2","DC5"):5, ("W2","DC6"):6,
    ("W3","DC1"):5, ("W3","DC2"):3, ("W3","DC3"):4, ("W3","DC4"):2, ("W3","DC5"):3, ("W3","DC6"):4
}

# Adjust costs for W2 by adding £1 per unit
cWD = {}
for (w,k), cost in cWD_original.items():
    if w == "W2":
        cWD[(w,k)] = cost + 1
    else:
        cWD[(w,k)] = cost

# Decision variables
xPW = {(p,w): LpVariable(f"xPW_{p}_{w}", lowBound=0) for p in P for w in W}
xWD = {(w,k): LpVariable(f"xWD_{w}_{k}", lowBound=0) for w in W for k in D}

# Model
model = LpProblem("SupplyChainOptimization_IncreasedCostsW2", LpMinimize)

# Objective
model += lpSum(cPW[(p,w)]*xPW[(p,w)] for p in P for w in W) + lpSum(cWD[(w,k)]*xWD[(w,k)] for w in W for k in D)

# Constraints

# Supply constraints
for p in P:
    model += lpSum(xPW[(p,w)] for w in W) <= supply[p], f"Supply_{p}"

# Demand constraints
for k in D:
    model += lpSum(xWD[(w,k)] for w in W) == demand[k], f"Demand_{k}"

# Flow balance at warehouses
for w in W:
    model += lpSum(xPW[(p,w)] for p in P) == lpSum(xWD[(w,k)] for k in D), f"WarehouseBalance_{w}"

# Solve
model.solve()

print("Status:", LpStatus[model.status])
print("Total Cost:", value(model.objective))

print("\nOptimal Shipments from Production to Warehouses:")
for p in P:
    for w in W:
        val = xPW[(p,w)].varValue
        if val > 0:
            print(f"{p} -> {w}: {val}")

print("\nOptimal Shipments from Warehouses to Distribution Centres:")
for w in W:
    for k in D:
        val = xWD[(w,k)].varValue
        if val > 0:
            print(f"{w} -> {k}: {val}")

Status: Optimal
Total Cost: 1820.0

Optimal Shipments from Production to Warehouses:
A -> W1: 40.0
A -> W2: 30.0
B -> W2: 90.0
C -> W3: 80.0
D -> W3: 60.0

Optimal Shipments from Warehouses to Distribution Centres:
W1 -> DC2: 40.0
W2 -> DC1: 50.0
W2 -> DC3: 60.0
W2 -> DC4: 10.0
W3 -> DC4: 60.0
W3 -> DC5: 50.0
W3 -> DC6: 30.0


In [9]:
#Task 4 (c)

In [11]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

P = ["A", "B", "C", "D"]
W = ["W1", "W2", "W3"]
D = ["DC1", "DC2", "DC3", "DC4", "DC5", "DC6"]

supply = {"A":70, "B":90, "C":80, "D":60}
demand = {"DC1":50, "DC2":40, "DC3":60, "DC4":70, "DC5":50, "DC6":30}

cPW = {
    ("A","W1"):4, ("A","W2"):3, ("A","W3"):5,
    ("B","W1"):6, ("B","W2"):2, ("B","W3"):4,
    ("C","W1"):5, ("C","W2"):4, ("C","W3"):3,
    ("D","W1"):7, ("D","W2"):5, ("D","W3"):6
}

cWD = {
    ("W1","DC1"):3, ("W1","DC2"):2, ("W1","DC3"):5, ("W1","DC4"):4, ("W1","DC5"):6, ("W1","DC6"):7,
    ("W2","DC1"):1, ("W2","DC2"):4, ("W2","DC3"):2, ("W2","DC4"):3, ("W2","DC5"):5, ("W2","DC6"):6,
    ("W3","DC1"):5, ("W3","DC2"):3, ("W3","DC3"):4, ("W3","DC4"):2, ("W3","DC5"):3, ("W3","DC6"):4
}

xPW = {(p,w): LpVariable(f"xPW_{p}_{w}", lowBound=0) for p in P for w in W}
xWD = {(w,k): LpVariable(f"xWD_{w}_{k}", lowBound=0) for w in W for k in D}

# Model
model = LpProblem("SupplyChainOptimization_W1Capacity", LpMinimize)

model += lpSum(cPW[(p,w)]*xPW[(p,w)] for p in P for w in W) + lpSum(cWD[(w,k)]*xWD[(w,k)] for w in W for k in D)


for p in P:
    model += lpSum(xPW[(p,w)] for w in W) <= supply[p], f"Supply_{p}"

for k in D:
    model += lpSum(xWD[(w,k)] for w in W) == demand[k], f"Demand_{k}"

for w in W:
    model += lpSum(xPW[(p,w)] for p in P) == lpSum(xWD[(w,k)] for k in D), f"WarehouseBalance_{w}"

model += lpSum(xPW[(p,"W1")] for p in P) <= 50, "Warehouse1Capacity"

model.solve()

print("Status:", LpStatus[model.status])
print("Total Cost:", value(model.objective))

print("\nOptimal Shipments from Production to Warehouses:")
for p in P:
    for w in W:
        val = xPW[(p,w)].varValue
        if val > 0:
            print(f"{p} -> {w}: {val}")

print("\nOptimal Shipments from Warehouses to Distribution Centres:")
for w in W:
    for k in D:
        val = xWD[(w,k)].varValue
        if val > 0:
            print(f"{w} -> {k}: {val}")


Status: Optimal
Total Cost: 1700.0

Optimal Shipments from Production to Warehouses:
A -> W1: 40.0
A -> W2: 30.0
B -> W2: 90.0
C -> W3: 80.0
D -> W2: 60.0

Optimal Shipments from Warehouses to Distribution Centres:
W1 -> DC2: 40.0
W2 -> DC1: 50.0
W2 -> DC3: 60.0
W2 -> DC4: 70.0
W3 -> DC5: 50.0
W3 -> DC6: 30.0


In [17]:
#Task 4 (d)

In [15]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

P = ["A", "B", "C", "D"]
W = ["W1", "W2", "W3"]
original_demand = {"DC1":50, "DC2":40, "DC3":60, "DC4":70, "DC5":50, "DC6":30}

supply = {"A":70, "B":90, "C":80, "D":60}

# Costs from Production to Warehouse
cPW = {
    ("A","W1"):4, ("A","W2"):3, ("A","W3"):5,
    ("B","W1"):6, ("B","W2"):2, ("B","W3"):4,
    ("C","W1"):5, ("C","W2"):4, ("C","W3"):3,
    ("D","W1"):7, ("D","W2"):5, ("D","W3"):6
}

# Costs from Warehouse to Distribution
cWD = {
    ("W1","DC1"):3, ("W1","DC2"):2, ("W1","DC3"):5, ("W1","DC4"):4, ("W1","DC5"):6, ("W1","DC6"):7,
    ("W2","DC1"):1, ("W2","DC2"):4, ("W2","DC3"):2, ("W2","DC4"):3, ("W2","DC5"):5, ("W2","DC6"):6,
    ("W3","DC1"):5, ("W3","DC2"):3, ("W3","DC3"):4, ("W3","DC4"):2, ("W3","DC5"):3, ("W3","DC6"):4
}

def solve_scenario(demand):
    # Create LP Model
    model = LpProblem("SupplyChainOptimization_RemoveDC", LpMinimize)
    
    # Decision variables
    xPW = {(p,w): LpVariable(f"xPW_{p}_{w}", lowBound=0) for p in P for w in W}
    xWD = {(w,k): LpVariable(f"xWD_{w}_{k}", lowBound=0) for w in W for k in demand.keys()}

    model += lpSum(cPW[(p,w)]*xPW[(p,w)] for p in P for w in W) + lpSum(cWD[(w,k)]*xWD[(w,k)] for w in W for k in demand if demand[k]>0)


    # Supply constraints
    for p in P:
        model += lpSum(xPW[(p,w)] for w in W) <= supply[p], f"Supply_{p}"

    # Demand constraints (only for DCs that have positive demand)
    for k in demand:
        if demand[k] > 0:
            model += lpSum(xWD[(w,k)] for w in W) == demand[k], f"Demand_{k}"
        else:
            # If a DC is removed, its demand is zero, ensure no supply goes there.
            model += lpSum(xWD[(w,k)] for w in W) == 0, f"Demand_{k}"

    # Warehouse balance constraints
    for w in W:
        model += lpSum(xPW[(p,w)] for p in P) == lpSum(xWD[(w,k)] for k in demand), f"WarehouseBalance_{w}"

    # Solve the model
    model.solve()
    
    solution = {
        'status': LpStatus[model.status],
        'objective': value(model.objective),
        'xPW': {key: xPW[key].varValue for key in xPW},
        'xWD': {key: xWD[key].varValue for key in xWD}
    }
    return solution

# Scenario 1: Remove DC1
demand_remove_DC1 = original_demand.copy()
demand_remove_DC1["DC1"] = 0
for k in ["DC2", "DC3", "DC4", "DC5", "DC6"]:
    demand_remove_DC1[k] += 10

sol_DC1 = solve_scenario(demand_remove_DC1)

# Scenario 2: Remove DC5
demand_remove_DC5 = original_demand.copy()
demand_remove_DC5["DC5"] = 0
for k in ["DC1", "DC2", "DC3", "DC4", "DC6"]:
    demand_remove_DC5[k] += 10

sol_DC5 = solve_scenario(demand_remove_DC5)

# Compare solutions
print("Scenario DC1 removed:")
print("Status:", sol_DC1['status'])
print("Total Cost:", sol_DC1['objective'])
print("Flows Production->Warehouse:", {k:v for k,v in sol_DC1['xPW'].items() if v>0})
print("Flows Warehouse->Distribution:", {k:v for k,v in sol_DC1['xWD'].items() if v>0})

print("\nScenario DC5 removed:")
print("Status:", sol_DC5['status'])
print("Total Cost:", sol_DC5['objective'])
print("Flows Production->Warehouse:", {k:v for k,v in sol_DC5['xPW'].items() if v>0})
print("Flows Warehouse->Distribution:", {k:v for k,v in sol_DC5['xWD'].items() if v>0})

# Determine best plan
if sol_DC1['objective'] < sol_DC5['objective']:
    print("\nBest Plan: Remove DC1")
    best_solution = sol_DC1
else:
    print("\nBest Plan: Remove DC5")
    best_solution = sol_DC5

print("\nChosen scenario objective cost:", best_solution['objective'])


Scenario DC1 removed:
Status: Optimal
Total Cost: 1820.0
Flows Production->Warehouse: {('A', 'W1'): 50.0, ('A', 'W2'): 20.0, ('B', 'W2'): 90.0, ('C', 'W3'): 80.0, ('D', 'W3'): 60.0}
Flows Warehouse->Distribution: {('W1', 'DC2'): 50.0, ('W2', 'DC3'): 70.0, ('W2', 'DC4'): 40.0, ('W3', 'DC4'): 40.0, ('W3', 'DC5'): 60.0, ('W3', 'DC6'): 40.0}

Scenario DC5 removed:
Status: Optimal
Total Cost: 1640.0
Flows Production->Warehouse: {('A', 'W1'): 50.0, ('A', 'W2'): 20.0, ('B', 'W2'): 90.0, ('C', 'W3'): 80.0, ('D', 'W2'): 60.0}
Flows Warehouse->Distribution: {('W1', 'DC2'): 50.0, ('W2', 'DC1'): 60.0, ('W2', 'DC3'): 70.0, ('W2', 'DC4'): 40.0, ('W3', 'DC4'): 40.0, ('W3', 'DC6'): 40.0}

Best Plan: Remove DC5

Chosen scenario objective cost: 1640.0
